In [ ]:
import numpy as np
import aadc

def sabr_normal_vol_atm(fwd, expiry, beta, sig0, rho, nu):
    F = fwd
    t = expiry
    t = np.where(abs(t) < 1e-10, 1e-10, t)

    c1 = 1 + ((2 - 3 * rho ** 2) / 24) * (nu ** 2) * t
    c2 = (rho * beta * nu * t) / (4 * F ** (1 - beta))
    c3 = beta * (beta - 2) * t / (24 * F ** (2 - 2 * beta))

    sig_n = (c1 * sig0 + c2 * sig0 ** 2 + c3 * sig0 ** 3) / (F ** (-beta))

    return sig_n


def sabr_normal_vol_otm(fwd, strike, expiry, beta, sig0, rho, nu):
    F = fwd
    K = strike
    t = expiry
    t = np.where(abs(t) < 1e-10, 1e-10, t)

    k = K / F
    alpha = sig0 / (F ** (1 - beta))

    beta_close_to_one = np.isclose(beta, 1, 1e-10)
    q = np.where(beta_close_to_one, np.log(k), (k ** (1 - beta) - 1) / (1 - beta))

    z = q * nu / alpha
    z_close_to_zero = np.isclose(z, 0, 1e-10)
    z = np.where(z_close_to_zero, np.nan, z)

    _H = z / np.log((np.sqrt(1 + 2 * rho * z + z ** 2) + z + rho) / (1 + rho))

    H = np.where(z_close_to_zero, 1, _H)

    _B = np.log((q * k ** (beta / 2)) / (k - 1)) * (alpha ** 2) / (q ** 2)
    _B += (rho / 4) * ((k ** beta - 1) / (k - 1)) * alpha * nu
    _B += ((2 - 3 * rho ** 2) / 24) * (nu ** 2)

    B = ((k - 1) / q) * (1 + _B * t)

    sig_n = sig0 * (F ** beta) * H * B

    return sig_n


def sabr_normal_vol(fwd, strike, expiry, beta, sig0, rho, nu):
    F, K, expiry, beta, sig0, rho, nu = np.broadcast_arrays(fwd, strike, expiry, beta, sig0, rho, nu)

    return np.where(np.isclose(F, K, 1e-6),
                     sabr_normal_vol_atm(F, expiry, beta, sig0, rho, nu),
                     sabr_normal_vol_otm(F, K, expiry, beta, sig0, rho, nu))


In [ ]:
ncalibrations = 1000

# Generate market data
fwd = np.linspace(0.01, 0.05, ncalibrations)
beta = np.tile([0.5, 0.6, 0.7, 0.8, 0.9], -(-ncalibrations // 5))[:ncalibrations]
expiry = np.tile(np.linspace(1, 10, 10), -(-ncalibrations // 10))[:ncalibrations]

np.random.seed(42)
sig0 = np.random.uniform(0.01, 0.05, ncalibrations)
rho = np.random.uniform(-0.5, 0., ncalibrations)
nu = np.random.uniform(0.2, 0.6, ncalibrations)

# 7 strikes, each row is an independent calibration scenario
strikes = np.linspace(0.25, 1.75, 7).reshape(-1, 1) * fwd
vols = sabr_normal_vol(fwd, strikes, expiry, beta, sig0, rho, nu)
strikes,vols

In [ ]:
from scipy.optimize import least_squares
vol_weights = [1., 1., 1., 100., 1., 1., 1.]

def residual(x, fwd, beta, expiry, strikes, vols):
    sig0, rho, nu = x
    rho = np.broadcast_to(rho, vols.shape)

    return np.where(np.abs(rho) > 0.9999,
                    np.ones_like(vols) * 1e6,
                    vol_weights * (sabr_normal_vol(fwd, strikes, expiry, beta, sig0, rho, nu) - vols))

x0 = np.array([0.02, -0.25, 0.03])

def sabr_normal_smile_fit(scen):
    results = least_squares(
        residual,
        x0,
        args=(fwd[scen], beta[scen], expiry[scen], strikes[:, scen], vols[:, scen]),
        method="lm",
        xtol=1e-6)

    return results.x

Solve for sig0, rho, nu for each scenario:

In [ ]:
sig0_bar, rho_bar, nu_bar = np.transpose([sabr_normal_smile_fit(j) for j in range(ncalibrations)])

rho_bar, nu_bar = np.where(nu_bar < 0, -rho_bar, rho_bar), np.abs(nu_bar)

In [ ]:
assert np.allclose(sig0_bar, sig0, atol=1e-10)
assert np.allclose(rho_bar, rho, atol=1e-10)
assert np.allclose(nu_bar, nu, atol=1e-10)
# sig0_bar - sig0, rho_bar - rho, nu_bar - nu
# np.argmax(np.abs(sig0_bar - sig0)), np.argmax(np.abs(rho_bar - rho)), np.argmax(np.abs(nu_bar - nu))


Now AADC the whole thing

In [ ]:
kernel = aadc.record(residual, x0, params=(fwd[0], beta[0], expiry[0], strikes[:, 0], vols[:, 0]), bump_size=1e-10)

In [ ]:
def sabr_normal_smile_fit_aadc(scen):
    kernel.set_params(fwd[scen], beta[scen], expiry[scen], strikes[:, scen], vols[:, scen])
    results = least_squares(
        kernel.func,
        x0,
        jac=kernel.jac,
        method="lm",
        xtol=1e-6)

    return results.x

Moment of truth:

In [ ]:
sig0_star, rho_star, nu_star = np.transpose([sabr_normal_smile_fit_aadc(j) for j in range(ncalibrations)])
rho_star, nu_star = np.where(nu_star < 0, -rho_star, rho_star), np.abs(nu_star)

assert np.allclose(sig0_bar, sig0_star, atol=1e-10)
assert np.allclose(rho_bar, rho_star, atol=1e-10)
assert np.allclose(nu_bar, nu_star, atol=1e-10)

# sig0_star - sig0_bar, rho_star - rho_bar, nu_star - nu_bar
# np.argmax(np.abs(sig0_star - sig0_bar)), np.argmax(np.abs(rho_star - rho_bar)), np.argmax(np.abs(nu_star - nu_bar))

